In [158]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import MiniBatchKMeans
from math import cos, asin, sqrt, pi
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import metrics
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sklearn.neighbors as neighbors
import copy

# Data Loading

In [159]:
bmtc = pd.read_parquet('./BMTC.parquet.gzip')
ip = pd.read_csv('./Input.csv',index_col=0)
gt = pd.read_csv('./GroundTruth.csv',index_col=0)

In [160]:
extracted_data = pd.read_csv('./extract.csv')
extracted_data_train = extracted_data[['Source_Lat','Source_Long','Dest_Lat','Dest_Long']]
extracted_data_label = extracted_data['TT'].values
# extracted_data = extracted_data.drop([])

# Preprocessing

In [161]:
def cluster_model(ip):
    coords = np.vstack((ip[['Source_Lat',  'Source_Long']].values,
                        ip[['Dest_Lat', 'Dest_Long']].values,
                        ip[['Source_Lat',  'Source_Long']].values,
                        ip[['Dest_Lat', 'Dest_Long']].values))
    nc = 300
    kmeans = MiniBatchKMeans(n_clusters=nc, batch_size=100).fit(coords)
    return kmeans

def haversine_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...
# lat1 = -34.83333#51.5007
# lon1 = -58.5166646#0.1246
# lat2 = 49.0083899664#40.6892
# lon2 = 2.53844117956#74.0445
# haversine_distance(lat1,lon1,lat2,lon2)
def preprocess(df):
    ip = copy.deepcopy(df)
    ip['haversine_distance'] = ip.apply(lambda x: haversine_distance(x['Source_Lat'],  x['Source_Long'],x['Dest_Lat'], x['Dest_Long']), axis=1)
    for i in range(nc):
        ip[f'pickup_cluster_{i+1}'] = np.zeros(len(ip))
        ip[f'dropoff_cluster{i+1}'] = np.zeros(len(ip))
    for index in ip.index:
        x  = ip.iloc[index]
        p1 = kmeans.predict([[x['Source_Lat'],  x['Source_Long']]])
        x[f'pickup_cluster_{p1}'] = 1
        p2 = kmeans.predict([[x['Dest_Lat'], x['Dest_Long']]])
        x[f'dropoff_cluster_{p2}'] = 1
#     ip['dropoff_cluster'] = ip.apply(lambda x: kmeans.predict([[x['Dest_Lat'], x['Dest_Long']]])[0], axis=1)
#     ip = pd.get_dummies(ip, columns = ['pickup_cluster','dropoff_cluster'])
    return ip

In [ ]:
given = preprocess(ip)
# # x_train, x_test, y_train, y_test = train_test_split(o.values,gt.values,train_size=0.8,test_size=0.2)
o = preprocess(extracted_data_train)
# x_train, x_test, y_train, y_test = train_test_split(extracted_data_train,extracted_data_label,train_size=0.8,test_size=0.2)
# frames = [given,o]
# combine = pd.concat(frames)
x_train, x_test, y_train, y_test = train_test_split(o,extracted_data_label,train_size=0.7,test_size=0.3,shuffle=True)


In [163]:

def score(y,y_pred):
    # Score
    mse = metrics.mean_squared_error(y, y_pred)
    mae = metrics.mean_absolute_error(y, y_pred)
    r2 = metrics.r2_score(y, y_pred)
    print(f'mae: {np.round(mae, 4)}') # mse:{np.round(mse, 4)}    r2: {np.round(r2, 4)}     
    return mae

def model_output(X_train, X_test, y_train, y_test):
    #Model Linear Regression
    reg = linear_model.LinearRegression()
    reg.fit(X_train, y_train)

    # Predict
    y_pred = reg.predict(X_train)
    print("1) Linear Regression ")
    print("Training Error:")
    score(y_train,y_pred)
    y_pred = reg.predict(X_test)
    print("Testing Error:")
    score(y_test,y_pred)
    
#     print("2) SVR")
#     regr = make_pipeline(StandardScaler(), SVR(C=1000, epsilon=0.01))
#     regr.fit(X_train, y_train)
#     # y_pred = regr.predict(one_hot_encoded_data)

#     # Predict
#     y_pred = regr.predict(X_train)
#     print("Training Error:")
#     score(y_train,y_pred)
#     y_pred = regr.predict(X_test)
#     print("Testing Error:")
#     score(y_test,y_pred)
    
    print("3) DT")
    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(X_train, y_train)

    # Predict
    y_pred = clf.predict(X_train)
    print("Training Error:")
    score(y_train,y_pred)
    y_pred = clf.predict(X_test)
    print("Testing Error:")
    score(y_test,y_pred)
    
    print("4) Ensemble model")
    # Training classifiers
    reg1 = GradientBoostingRegressor(random_state=1)
    reg2 = neighbors.KNeighborsRegressor(17)#RandomForestRegressor(random_state=1)
    reg3 = RandomForestRegressor(random_state=1,max_depth=112,n_estimators=1733)
    reg4 = make_pipeline(StandardScaler(), SVR(C=1, epsilon=0.01))
    ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2),('rf2',reg3),])#('svr',reg4)])
    ereg = ereg.fit(X_train, y_train)


    # Predict
    y_pred = ereg.predict(X_train)
    print("Training Error:")
    score(y_train,y_pred)
    y_pred = ereg.predict(X_test)
    print("Testing Error:")
    score(y_test,y_pred)

    n_neighbors=17
    print("5) KNN")
    knn = neighbors.KNeighborsRegressor(n_neighbors)
    knn = knn.fit(X_train, y_train)
    # Predict
    y_pred = knn.predict(X_train)
    print("Training Error:",sep = '')
    score(y_train,y_pred)
    y_pred = knn.predict(X_test)
    print("Testing Error:")
    score(y_test,y_pred)


In [170]:
model_output(x_train, x_test, y_train, y_test)

1) Linear Regression 
Training Error:
mae: 4.6939
Testing Error:
mae: 4.6251
3) DT
Training Error:
mae: 0.0174
Testing Error:
mae: 2.2391
4) Ensemble model
Training Error:
mae: 1.6333
Testing Error:
mae: 2.1013
5) KNN
Training Error:
mae: 2.1707
Testing Error:
mae: 2.2882


In [169]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    print('Model Performance')
    mapp = score(test_labels,predictions)#np.mean(errors)
    return mapp
# Training classifiers
reg1 = GradientBoostingRegressor(random_state=1)
reg2 = neighbors.KNeighborsRegressor(11)#RandomForestRegressor(random_state=1)
reg3 = RandomForestRegressor(random_state=1,max_depth=112,n_estimators=1733)
# reg4 = make_pipeline(StandardScaler(), SVR(C=1, epsilon=0.01))
ereg = VotingRegressor(estimators=[ ('gb', reg1),('rf', reg2),('rf2',reg3),])#('svr',reg4)]) 
ereg = ereg.fit(x_train,y_train)
t=evaluate(ereg,given.values,gt.values)
t=evaluate(ereg,x_test,y_test)
t=evaluate(ereg,x_train,y_train)

Model Performance
mae: 18.9569
Model Performance
mae: 2.0959
Model Performance
mae: 1.6


In [143]:
import pickle 
with open('ensemble','wb') as file:
    pickle.dump(ereg,file)
with open('ensemble','rb') as file:
    model = pickle.load(file)

In [164]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(x_train, y_train)

# Predict
y_pred = clf.predict(x_train)
print("Training Error:")
score(y_train,y_pred)
y_pred = clf.predict(x_test)
print("Testing Error:")
score(y_test,y_pred)
y_pred = clf.predict(given.values)
print("Real Testing Error:")
score(gt.values,y_pred)

Training Error:
mae: 0.0174
Testing Error:
mae: 2.2685
Real Testing Error:
mae: 18.7884


18.788409405255877

In [166]:
rf_model = RandomForestRegressor(random_state=1,max_depth=112,n_estimators=1733)
rf_model = rf_model.fit(x_train, y_train)

# Predict
y_pred = rf_model.predict(x_train)
print("Training Error:")
score(y_train,y_pred)
y_pred = rf_model.predict(x_test)
print("Testing Error:")
score(y_test,y_pred)
y_pred = rf_model.predict(given.values)
print("Real Testing Error:")
score(gt.values,y_pred)

Training Error:
mae: 0.7352
Testing Error:
mae: 1.9409
Real Testing Error:
mae: 18.9815


18.98146656035835

In [165]:
import xgboost as xg
xgb_r = xg.XGBRegressor(objective ='reg:linear',n_estimators=1788,max_depth=112, seed = 123)

# Fitting the model
xgb_r.fit(x_train, y_train)
 
# Predict
y_pred = xgb_r.predict(x_train)
print("Training Error:")
score(y_train,y_pred)
y_pred = xgb_r.predict(x_test)
print("Testing Error:")
score(y_test,y_pred)
y_pred = xgb_r.predict(given.values)
print("Real Testing Error:")
score(gt.values,y_pred)


[19:31:31] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
Training Error:
mae: 0.0178
Testing Error:
mae: 2.0503
Real Testing Error:
mae: 18.8795


18.87949205631207

In [155]:
!pip install xgboost

     |████████████████████████████████| 173.6 MB 6.0 kB/s            


# KNN

In [172]:
# from sklearn.model_selection import GridSearchCV

parameters={'n_neighbors':[11]}
print("5) KNN")
knn = GridSearchCV(estimator=neighbors.KNeighborsRegressor(),param_grid=parameters)
knn = knn.fit(x_train,y_train)
print(knn.best_params_)
knn = knn.best_estimator_ # 11 is best param
# Predict
y_pred = knn.predict(x_train)
print("Training Error:",sep = '')
score(y_train,y_pred)
y_pred = knn.predict(x_test)
print("Testing Error:")
score(y_test,y_pred)
print("Real Testing Error:",given.values.shape)
y_pred = knn.predict(given.values)
score(y_pred,gt.values)

5) KNN
{'n_neighbors': 11}
Training Error:
mae: 2.0799
Testing Error:
mae: 2.2753
Real Testing Error: (1205, 605)
mae: 18.4561


18.456113416320882